<a href="https://colab.research.google.com/github/PrAsAnNaRePo/chat-ft/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.__version__

'2.6.0+cu124'

In [ ]:
!pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision


**install unsloth**
- open [unsolth repo](https://github.com/unslothai/unsloth?tab=readme-ov-file) to check what version to is installed.
- for my setup i have `torch 2.5.1` and `cuda 12.1`, look for the specific version of unsolth package.

In [ ]:
%pip install --upgrade pip
%pip install "unsloth[cu121-torch250] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4di9hok3/unsloth_ddabc9319cf94b32ba93e6442c18912d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4di9hok3/unsloth_ddabc9319cf94b32ba93e6442c18912d
  Resolved https://github.com/unslothai/unsloth.git to commit 038e6d4c8d40207a87297ab3aaf787c19b1006d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 98.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

> **NOTE**:
> please make sure to restart the session inoder to use the latest dependicies we've instaled.

**prepare the code**
- clone the repo by using `git clone https://github.com/PrAsAnNaRePo/chat-ft.git`

In [ ]:
!git clone https://github.com/PrAsAnNaRePo/chat-ft.git

Cloning into 'chat-ft'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 23 (delta 10), reused 19 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 11.00 KiB | 563.00 KiB/s, done.
Resolving deltas: 100% (10/10), done.


**import dataset**
- export a chat of your interest in th whatsapp, this will gives you the `.txt` file.
- use the file as the dataset to train the model.

In [ ]:
# set the wandb for logging
import os

os.environ["WANDB_API_KEY"] = "your_api_key_here"

In [ ]:
# login to HF to push models to the repo
!pip install -U "huggingface_hub[cli]"
!huggingface-cli login --token "your_api_key_here" # get you access token from https://huggingface.co/settings/tokens

  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `chat-ft-tutorial` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `chat-ft-tutorial`


In [ ]:
!cd chat-ft && python train.py \
  --chat_file_path /kaggle/input/chat-file/raw_chat.txt \
  --target_role "<your target chat name>" \
  --hr_gap 3 \
  --low_gpu_memory True \
  --output_path ./unsloth_qwen-chat-ft \
  --num_epochs 4 \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 4 \
  --wandb_logger True

2025-02-02 17:28:07.330213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-02 17:28:07.352263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-02 17:28:07.358759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Figure(1200x600)
Figure(1200x600)
Figure(1600x600)
Lenght of dataset:  509
==((====))==  Unsloth 2025.1.8: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = 'Qwen/Qwen2.5-1.5B-Instruct'
adapter_model = '/kaggle/working/chat-ft/unsloth_qwen-chat-ft/checkpoint-64'
prompt = "what is your name"

model = AutoModelForCausalLM.from_pretrained(base_model, device_map='auto')
model = PeftModel.from_pretrained(model, adapter_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
                (ba

In [ ]:

tokenized = tokenizer.apply_chat_template(
    [
        {
            'role': 'system',
            'content': 'a conversation between interesting peoples'
        },
        {
            'role': 'user',
            'content': "hey, what up!"
        }
    ],
    tokenize=False
)

tokens = tokenizer(tokenized, return_tensors='pt').to('cuda')

res = model.generate(**tokens, max_new_tokens=150)
print(tokenizer.decode(res.reshape(-1)))

<|im_start|>system
a conversation between interesting peoples<|im_end|>
<|im_start|>user
hey, what up!<|im_end|>
<|im_start|>assistant
Nothing much
U?<|im_end|>
